In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import time

In [11]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [12]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [28]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_sigmoid_stack = nn.Sequential(
            nn.Linear(28*28, 84),
            nn.ReLU(),
            nn.Linear(84, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_sigmoid_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_sigmoid_stack): Sequential(
    (0): Linear(in_features=784, out_features=84, bias=True)
    (1): ReLU()
    (2): Linear(in_features=84, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
    (5): Softmax(dim=None)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [32]:
epochs = 5
start_time = time.time()
for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
end_time = time.time()
print("Done!")
print(f"Training time: {end_time-start_time:.2f} seconds")

loss: 2.299937  [   64/60000]
loss: 2.291889  [ 6464/60000]
loss: 2.310803  [12864/60000]
loss: 2.309150  [19264/60000]
loss: 2.308951  [25664/60000]
loss: 2.304678  [32064/60000]
loss: 2.311326  [38464/60000]
loss: 2.293667  [44864/60000]
loss: 2.307395  [51264/60000]
loss: 2.325466  [57664/60000]
Test Error: 
 Accuracy: 8.9%, Avg loss: 2.306398 

loss: 2.299937  [   64/60000]
loss: 2.291889  [ 6464/60000]
loss: 2.310803  [12864/60000]
loss: 2.309150  [19264/60000]
loss: 2.308951  [25664/60000]
loss: 2.304678  [32064/60000]
loss: 2.311326  [38464/60000]
loss: 2.293667  [44864/60000]
loss: 2.307395  [51264/60000]
loss: 2.325466  [57664/60000]
Test Error: 
 Accuracy: 8.9%, Avg loss: 2.306398 

loss: 2.299937  [   64/60000]
loss: 2.291889  [ 6464/60000]
loss: 2.310803  [12864/60000]
loss: 2.309150  [19264/60000]
loss: 2.308951  [25664/60000]
loss: 2.304678  [32064/60000]
loss: 2.311326  [38464/60000]
loss: 2.293667  [44864/60000]
loss: 2.307395  [51264/60000]
loss: 2.325466  [57664/60000

In [30]:
# Define model
class LetNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(4,4), stride=(1,1), padding=(0,0))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(4,4), stride=(1,1), padding=(0,0))
        self.conv3 = nn.Conv2d(16, 120, kernel_size=(4,4), stride=(1,1), padding=(0,0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = LetNet().to(device)

In [25]:
x = torch.randn(64, 1, 28, 28)
model = LetNet()
print(model(x).shape)

torch.Size([64, 10])
